# Projet 4 : Anticiper les besoins en consommation électrique de bâtiments
*Pierre-Eloi Ragetly*

Ce projet fait parti du parcours *DataScientist* d'OpenClassrooms.

L'objectif principal est de trouver un modèle permettant de prédire **les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l'habitation.**

Pour cela nous disposons des données de la ville de Seattle pour les années 2015 et 2016. Ces données sont à récupérer sur le site kaggle.

# Partie III : Data modeling

Ce notebook a pour but de présenter le travail effectué sur la modélisation.

Nous commencerons par séparer notre jeu de données en deux parties distinctes:
- Le **training set**, qui va permettre d'entrainer les différents modèles;
- Le **testing set**, qui permettra de déterminer la performance du modèle finale.

Pour ce faire, la méthode `train_test_split()` de la classe *sklearn.model_selection* sera utilisée en réservant 20% des données pour le jeu de test.

Puis les modèles les plus courants seront entraînés et comparés afin de conserver les plus prometteurs. Au préalable, *une recherche par quadrillage* sera effectuée pour automatiser le choix des *hyperparamètres*, et les variables les plus pertinentes seront sélectionnées par **RFE** (Recursive Feature Elimination).

Après sélection des modèles les plus performants, nous affinerons encore les hyperparamètres à l'aide d'une *recherche aléatoire* cette fois ci, et nous en profiterons pour tester la pertinence de la variable *EnergyStarScore*.

Nous analyserons enfin les erreurs des modèles afin de déterminer s'il est pertinent d'utiliser une *méthode d'ensemble*, ie. combiner plusieurs modèles pour construire un modèle plus performant.

Le modèle final obtenu, nous pourrons évaluer sa performance à l'aide du jeu de test.

In [1]:
# Import des librairies usuelles
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import seaborn as sns

In [2]:
# Change some default parameters of matplotlib using seaborn
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({'axes.titleweight': 'bold'})
sns.set(style='ticks')
current_palette = sns.color_palette('RdBu')
sns.set_palette(current_palette)

In [3]:
# import data
data = (pd.read_csv('data/data_tr.csv')
          .set_index('OSEBuildingID')
          .drop(columns='ENERGYSTARScore'))
data_star = (pd.read_csv('data/data_tr.csv')
              .set_index('OSEBuildingID')
              .dropna(subset=['ENERGYSTARScore']))

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modéliser-la-consommation-totale-d’énergie" data-toc-modified-id="Modéliser-la-consommation-totale-d’énergie-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modéliser la consommation totale d’énergie</a></span><ul class="toc-item"><li><span><a href="#Créer-un-jeu-de-test" data-toc-modified-id="Créer-un-jeu-de-test-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Créer un jeu de test</a></span></li><li><span><a href="#Comparaison-des-modèles" data-toc-modified-id="Comparaison-des-modèles-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Comparaison des modèles</a></span><ul class="toc-item"><li><span><a href="#En-conservant-les-valeurs-par-défaut-des-hyperparamères" data-toc-modified-id="En-conservant-les-valeurs-par-défaut-des-hyperparamères-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>En conservant les valeurs par défaut des hyperparamères</a></span></li><li><span><a href="#En-optimisant-les-hyperparamètres-via-des-recherches-aléatoires-et/ou-par-grille" data-toc-modified-id="En-optimisant-les-hyperparamètres-via-des-recherches-aléatoires-et/ou-par-grille-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>En optimisant les hyperparamètres via des recherches aléatoires et/ou par grille</a></span></li></ul></li><li><span><a href="#Recherche-des-variables-les-plus-significatives" data-toc-modified-id="Recherche-des-variables-les-plus-significatives-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Recherche des variables les plus significatives</a></span></li><li><span><a href="#Sélection-des-cinq-meilleurs-modèles" data-toc-modified-id="Sélection-des-cinq-meilleurs-modèles-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Sélection des cinq meilleurs modèles</a></span></li><li><span><a href="#Modèle-final" data-toc-modified-id="Modèle-final-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Modèle final</a></span></li><li><span><a href="#Vérifier-la-pertinence-de-Energy-Star-Score" data-toc-modified-id="Vérifier-la-pertinence-de-Energy-Star-Score-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Vérifier la pertinence de Energy Star Score</a></span></li></ul></li><li><span><a href="#Modéliser-la-consommation-totale-d’énergie" data-toc-modified-id="Modéliser-la-consommation-totale-d’énergie-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modéliser la consommation totale d’énergie</a></span><ul class="toc-item"><li><span><a href="#Créer-un-jeu-de-test" data-toc-modified-id="Créer-un-jeu-de-test-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Créer un jeu de test</a></span></li><li><span><a href="#Comparaison-des-modèles" data-toc-modified-id="Comparaison-des-modèles-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Comparaison des modèles</a></span></li><li><span><a href="#Recherche-des-variables-les-plus-significatives" data-toc-modified-id="Recherche-des-variables-les-plus-significatives-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Recherche des variables les plus significatives</a></span></li><li><span><a href="#Sélection-des-cinq-meilleurs-modèles" data-toc-modified-id="Sélection-des-cinq-meilleurs-modèles-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Sélection des cinq meilleurs modèles</a></span></li><li><span><a href="#Modèle-final" data-toc-modified-id="Modèle-final-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Modèle final</a></span></li><li><span><a href="#Vérifier-la-pertinence-de-Energy-Star-Score" data-toc-modified-id="Vérifier-la-pertinence-de-Energy-Star-Score-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Vérifier la pertinence de Energy Star Score</a></span></li></ul></li></ul></div>

## Modéliser la consommation totale d’énergie

Pour pouvoir modéliser la consommation totale en énergie il faut au préalable déterminer quelle sera notra valeur cible. Sous nous avons quatre variables potentielles :
- SiteEnergyUse(kBtu)
- SiteEnergyUseWN(kBtu)
- SiteEnergyUse(kBtu)_log
- SiteEnergyUseWN(kBtu)_log

Nous avons vu lors de l'ingénierie des variable qu'il était préférable de prendre la version log pour avoir une distribution se rapprochant d'une distribution normale. On peut donc déjà écarter les deux premières.

Se pose ensuite la question de savoir s'il est préférable de garder la version normalisée ou non normalisée. Pour rappel, la version normalisée et la consommation corrigée en prenant comme référence la température des trentes dernières années. Alors que la version normalisée est la consommation moyenne sur les années 2015 et 2016. Dans le contexte de réchauffement climatique, il est fort à parier que la températures des prochaines années sera plus proche de celles de 2015 et 2016 que de la température des trentes dernières années.

Nous prendrons donc la version non normalisée **SiteEnergyUse(kBtu)_log**.

### Créer un jeu de test

In [4]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, :-6].values
y = data.loc[:, 'SiteEnergyUse(kBtu)_log'].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

Le paramètre *random_state* permet de définir *le germe* (seed) du générateur de nombre aléatoires, afin qu'il génère toujours la même suite d'indices pseudo-aléatoires.

### Comparaison des modèles

Pour comparer les résultats obtenus avec les algorithmes de regression les plus courants (voir liste ci-dessous), nous utiliserons la librairie *Scikit-Learn* ainsi que la librairie *XGBoost*.
- Régression Ridge
- Régression Lasso
- Elastic Net
- Régression SVM linéaire
- Régression SVM avec noyau
- Régression kNN
- Arbre de décision
- Forêt aléatoire
- Gradient Boosting
- XGBoost
- Perceptron multi-couches

Nous utiliserons comme mesure de performance la RMSE par validation croisée.

#### En conservant les valeurs par défaut des hyperparamères

In [5]:
from functions.ml_modeling import get_models
from functions.ml_modeling import compare_models

# Create models
default_models = get_models(X_train, y_train)

# Compare models
compare_models(X_train, y_train, default_models)

,RMSE,RMSE_std,R2,R2_std
Ridge,1.271064,0.815615,-0.287782,1.675330
Lasso,1.301737,0.037429,-0.000678,0.000595
ElasticNet,1.281750,0.044597,0.030058,0.015660
LinearSVR,1.348145,0.788955,-0.385276,1.642602
SVR,0.833117,0.036242,0.588541,0.040997
KNeighborsRegressor,0.991973,0.044768,0.414600,0.077137
DecisionTreeRegressor,0.871922,0.049904,0.549237,0.052139
RandomForestRegressor,0.647185,0.011835,0.752250,0.012061
GradientBoostingRegressor,0.626153,0.019641,0.768118,0.013423
XGBRegressor,0.651344,0.020638,0.748912,0.017389


#### En optimisant les hyperparamètres via des recherches aléatoires et/ou par grille

In [6]:
# Create models
models = get_models(X_train, y_train, best_hparams=True)

# Compare models
df = compare_models(X_train, y_train, models)
df

,RMSE,RMSE_std,R2,R2_std
Ridge,0.954890,0.052254,0.460463,0.049179
Lasso,1.146331,0.135182,0.219630,0.158305
ElasticNet,0.959244,0.056865,0.455671,0.051165
LinearSVR,1.298493,0.353942,-0.054854,0.571335
SVR,0.780573,0.043703,0.637942,0.048712
KNeighborsRegressor,0.973904,0.040785,0.438066,0.051661
DecisionTreeRegressor,0.734815,0.024689,0.680798,0.016623
RandomForestRegressor,0.641131,0.013494,0.756885,0.011884
GradientBoostingRegressor,0.616372,0.014684,0.774813,0.019057
XGBRegressor,0.615812,0.019145,0.775576,0.015342


### Recherche des variables les plus significatives

Une grande qualité des forêts aléatoires est qu'elles permettent de mesurer facilement l'importance relative des variables. C'est donc tout naturellement que nous allons utiliser cet algorithme pour sélectionner les variables les plus pertinentes.

Pour cela nous allons utiliser la classe **RFECV** de *sklearn.feature_selection*. La méthode RFE (Recursive Feature Elimination) permet de supprimer les variables de manière récursives. Le modèle est d'abord entrainé sur toutes les variables, puis l'algorithme écarte à chaque itération la ou les variables (depend du pas renseigné par l'utilisateur) les moins significatives jusqu'à obtenir le nombre de variable voulu. Pour déterminer le nombre de variables à conserver, on utilise une validation croisée.

Ensuite nous ne conserverons que les variables qui sont significatives et réentrainerons les modèles avec ces variables.

Nous observons que les meilleurs résultats sont obtenus lorsque nous conservons 29 variables sur les 54. Regardons ce qu'il se passe quand on entraine les modèles sur 29 variables.

In [7]:
from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import RFECV

cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
min_features = int(X_train.shape[1]/5)
forest_reg = [m for m in models if type(m).__name__=='RandomForestRegressor'][0]

rfe = RFECV(forest_reg, min_features_to_select=min_features,
            cv=cv)
rfe.fit(X_train, y_train)
X_train_rfe = rfe.transform(X_train)
mask = rfe.support_

print('{} features have been selected'.format(rfe.n_features_))

54 features have been selected


Nous observons que les meilleurs résultats sont obtenus lorsque nous conservons 48 variables sur les 54. Regardons ce qu'il se passe quand on entraine les modèles sur 48 variables.

In [8]:
# Create models
models_rfe = get_models(X_train_rfe, y_train, best_hparams=True)

# Compare models
df_rfe = compare_models(X_train_rfe, y_train, models_rfe)
df_rfe

,RMSE,RMSE_std,R2,R2_std
Ridge,0.954890,0.052254,0.460463,0.049179
Lasso,1.146331,0.135182,0.219630,0.158305
ElasticNet,0.959244,0.056865,0.455671,0.051165
LinearSVR,1.298493,0.353942,-0.054854,0.571335
SVR,0.780573,0.043703,0.637942,0.048712
KNeighborsRegressor,0.973904,0.040785,0.438066,0.051661
DecisionTreeRegressor,0.734815,0.024689,0.680798,0.016623
RandomForestRegressor,0.641131,0.013494,0.756885,0.011884
GradientBoostingRegressor,0.616372,0.014684,0.774813,0.019057
XGBRegressor,0.615812,0.019145,0.775576,0.015342


Les résultats sont similaires, voir légèrement meilleurs pour certains modèles. Nous continuerons donc avec 29 variables, cela permettra de réduire les temps d'entrainement sans abaisser la performance pour autant.

### Sélection des cinq meilleurs modèles

In [9]:
# Get a score based on both RMSE and R2 scores
df_rfe['RMSE_minmax'] = (df_rfe['RMSE']-df_rfe['RMSE'].min())/ \
                        (df_rfe['RMSE'].max()-df_rfe['RMSE'].min())
df_rfe['R2_minmax'] = (df_rfe['R2']-df_rfe['R2'].min())/ \
                           (df_rfe['R2'].max()-df_rfe['R2'].min())
df_rfe['Score'] = df_rfe['R2_minmax'] - df_rfe['RMSE_minmax']

# Keep the 3 models with the largest score
best_models_name = df_rfe['Score'].nlargest(5).index
best_models = [m for m in models_rfe if type(m).__name__ in best_models_name]
print("The top five models are:")
for m in best_models_name:
    print(m)

The top five models are:
XGBRegressor
GradientBoostingRegressor
RandomForestRegressor
DecisionTreeRegressor
MLPRegressor


### Modèle final

Maintenant que nous avons réentrainé les modèles les plus prometteurs en ne conservant que les variables significatives, nous allons pouvoir obtenir le modèle finale.

Pour ce modèle finale, nous allons reprendre l'astuce des modèles ensemblistes, nous allons chercher à obtenir le meilleur des modèles sélectionnés en les empilant de manière à former un nouveau modèle plus performant. Pour cela, nous utiliserons la classe **StackingRegressor** de *sklearn.ensemble*.

In [10]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

# Get a stacking ensemble of models
estimators = []
for m in best_models:
    estimators.append((type(m).__name__, m))
stack_reg = StackingRegressor(estimators=estimators,
                              final_estimator=RidgeCV(alphas=np.logspace(-2, 2, 5)))

# Get the results
best_models.append(stack_reg)
df = compare_models(X_train_rfe, y_train, best_models)
df

,RMSE,RMSE_std,R2,R2_std
DecisionTreeRegressor,0.734815,0.024689,0.680798,0.016623
RandomForestRegressor,0.641131,0.013494,0.756885,0.011884
GradientBoostingRegressor,0.616372,0.014684,0.774813,0.019057
XGBRegressor,0.615812,0.019145,0.775576,0.015342
MLPRegressor,0.755391,0.046810,0.660108,0.052493
StackingRegressor,0.605804,0.015737,0.782378,0.019733


Le modèle ainsi créé est plus performant. Cependant, le gain de performance est bien faible comparé aux résultats obtenus avec le XGBoost, alors que le temps de calcul nécessaire est lui nettement supérieur (il faut entrainer six modèles au lieu d'un seul). C'est pour cette raison que le modèle XGBoost est préféré au modèle qui utilise la technique de l'empilement.

Pour finir, regardons ce que donne le XGBoost sur les données de tests.

In [14]:
from sklearn.metrics import r2_score

gboost = best_models[-4]
y_pred = np.exp(gboost.predict(X_test[:, mask]))
y_true = np.exp(y_test)
r2 = r2_score(y_true, y_pred)
print(r2)

0.9112227338597874


### Vérifier la pertinence de Energy Star Score

Nous allons pour clore cette partie consacrée à la modélisation des consommations énergétiques, vérifier la pertinence de la variable *Energy Star Score*. Pour cela nous allons entrainer le modèle XGBoost sur deux jeux données distincts uniquement par la présence (ou non) de cette variable. Puis nous comparerons les résultats obtenus avec chaque modèles.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from functions.ml_modeling import gboost_reg_best_params


# Split the dataset into random train and test subsets
mask_star = (data_star.columns[:-6]!='ENERGYSTARScore')
X_star = data_star.iloc[:, :-6].values
y_star = data_star.loc[:, 'SiteEnergyUse(kBtu)_log'].values
X_star_train, X_star_test, y_star_train, y_star_test = train_test_split(X_star,
                                                                        y_star,
                                                                        test_size=0.2,
                                                                        random_state=42)
# Train the models
gboost_star = gboost_reg_best_params(X_star_train,
                                     y_star_train,
                                     GradientBoostingRegressor())
gboost_no_star = gboost_reg_best_params(X_star_train[:, mask_star],
                                        y_star_train,
                                        GradientBoostingRegressor())

# Get results
r2 = []
y_pred = np.exp(gboost_star.predict(X_star_test))
y_true = np.exp(y_star_test)
r2.append(r2_score(y_true, y_pred))
y_pred = np.exp(gboost_no_star.predict(X_star_test[:, mask_star]))
r2.append(r2_score(y_true, y_pred))
df_star = pd.DataFrame({'EnergyStarScore': [True, False], 'R2': r2})
df_star

On constate que l'Energy Star Score permet bien d'améliorer le modèle. Cependant, bon nombre de bâtiments n'ont pas ce score. Or le résultat du 1.5 montre clairement qu'il est préférable de conserver un maximum de données, quitte à se passer de l'Energy Star Score.

## Modéliser la consommation totale d’énergie

Là encore, nous utiliserons la version log pour l'entrainement des modèles

### Créer un jeu de test

In [ ]:
from sklearn.model_selection import train_test_split

X = data.iloc[:, :-6].values
y = data.loc[:, 'TotalGHGEmissions_log'].values
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

### Comparaison des modèles

In [ ]:
# Create models
models = get_models(X_train, y_train, best_hparams=True)

# Compare models
df = compare_models(X_train, y_train, models)
df

### Recherche des variables les plus significatives

In [ ]:
cv = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)
min_features = int(X_train.shape[1]/5)
forest_reg = [m for m in models if type(m).__name__=='RandomForestRegressor'][0]

rfe = RFECV(forest_reg, min_features_to_select=min_features,
            cv=cv)
rfe.fit(X_train, y_train)
X_train_rfe = rfe.transform(X_train)
mask = rfe.support_

print('{} features have been selected'.format(rfe.n_features_))

In [ ]:
# Create models
models_rfe = get_models(X_train_rfe, y_train, best_hparams=True)

# Compare models
df_rfe = compare_models(X_train_rfe, y_train, models_rfe)
df_rfe

### Sélection des cinq meilleurs modèles

In [ ]:
# Get a score based on both RMSE and R2 scores
df_rfe['RMSE_minmax'] = (df_rfe['RMSE']-df_rfe['RMSE'].min())/ \
                        (df_rfe['RMSE'].max()-df_rfe['RMSE'].min())
df_rfe['R2_minmax'] = (df_rfe['R2']-df_rfe['R2'].min())/ \
                           (df_rfe['R2'].max()-df_rfe['R2'].min())
df_rfe['Score'] = df_rfe['R2_minmax'] - df_rfe['RMSE_minmax']

# Keep the 3 models with the largest score
best_models_name = df_rfe['Score'].nlargest(5).index
best_models = [m for m in models_rfe if type(m).__name__ in best_models_name]
print("The top five models are:")
for m in best_models_name:
    print(m)

### Modèle final

In [ ]:
# Get a stacking ensemble of models
estimators = []
for m in best_models:
    estimators.append((type(m).__name__, m))
stack_reg = StackingRegressor(estimators=estimators,
                              final_estimator=RidgeCV(alphas=np.logspace(-2, 2, 5)))

# Get the results
best_models.append(stack_reg)
df = compare_models(X_train_rfe, y_train, best_models)
df

In [ ]:
gboost = best_models[-4]
y_pred = np.exp(gboost.predict(X_test[:, mask]))
y_true = np.exp(y_test)
r2 = r2_score(y_true, y_pred)
print(r2)

### Vérifier la pertinence de Energy Star Score

In [ ]:
# Split the dataset into random train and test subsets
mask_star = (data_star.columns[:-6]!='ENERGYSTARScore')
X_star = data_star.iloc[:, :-6].values
y_star = data_star.loc[:, 'TotalGHGEmissions_log'].values
X_star_train, X_star_test, y_star_train, y_star_test = train_test_split(X_star,
                                                                        y_star,
                                                                        test_size=0.2,
                                                                        random_state=42)
# Train the models
gboost_star = gboost_reg_best_params(X_star_train,
                                     y_star_train,
                                     GradientBoostingRegressor())
gboost_no_star = gboost_reg_best_params(X_star_train[:, mask_star],
                                        y_star_train,
                                        GradientBoostingRegressor())

# Get results
r2 = []
y_pred = np.exp(gboost_star.predict(X_star_test))
y_true = np.exp(y_star_test)
r2.append(r2_score(y_true, y_pred))
y_pred = np.exp(xgboost_no_star.predict(X_star_test[:, mask_star]))
r2.append(r2_score(y_true, y_pred))
df_star = pd.DataFrame({'EnergyStarScore': [True, False], 'R2': r2})
df_star